# Reading real-time EEG data from the LSL server

TODO: Authors

TODO: Intro to reading in epochs

TODO: based on notebook from mne_realtime + bitbrain script

In [2]:
import mne
from mne_realtime import LSLClient, MockLSLStream
from flask_sse import sse
import time
import random
import bbtools as bbt
import numpy as np
#from mne.datasets import sample
#from mne.io import read_raw_fif
# this is the max wait time in seconds until client connection
#wait_max = 5

In [3]:
# TODO: replace this with info object alone

raw = bbt.read_csv("/Users/Javier/Artificial intelligence/AI6 3rd edition/brain-controlled-game/Mock_lsl_for_testing/Datasets/EEG-02-08-jv_baseline_eyes_open.csv",
                   ['Fp1', 'Fp2', 'F3', 'F4', 'C1', 'C3', 'C2', 'C4', 'CP1', 'CP3', 'CP2', 'CP4', 'Cz', 'O1', 'O2', 'Pz'])

Creating RawArray with float64 data, n_channels=16, n_times=60712
    Range : 0 ... 60711 =      0.000 ...   237.152 secs
Ready.


In [4]:
def process_epoch(epoch):
    # IMPLEMENT HERE ARTIFACT DETECTION LOGIC
    #me llega el lsl en formato epoc, no en formato raw. Hay que ajustar el código para que epoch se convierta en raw
#usar funcion epoch.get_data()
    #non_frontal_channels = ['C1', 'C2', 'C3', 'C4', 'CP1', 'CP2', 'CP3', 'CP4', 'O1', 'O2', 'Cz', 'Pz']
    #epoch = epoch.drop_channels(non_frontal_channels)
    #print (epoch)
    raw = epoch.get_data()
    #print (raw)
    range1 = np.amax(raw[0][0]) - np.amin(raw[0][0])
    range2 = np.amax(raw[0][1]) - np.amin(raw[0][1])
    range3 = np.amax(raw[0][2]) - np.amin(raw[0][2])
    range4 = np.amax(raw[0][3]) - np.amin(raw[0][3])
    signal_range = (range1+range2+range3+range4)/4
    signal_out4=np.amax(raw[0][4])-np.amin(raw[0][4])
    signal_out6=np.amax(raw[0][6])-np.amin(raw[0][6])
    signal_out8=np.amax(raw[0][8])-np.amin(raw[0][8])
    signal_out10=np.amax(raw[0][10])-np.amin(raw[0][10])
    signal_out_range = (signal_out4 + signal_out6 + signal_out8 + signal_out10)/4

    print ("average range= "+str(signal_range))

    #print (signal_range)
    # Use send_blink_artifact_event() to send the event to the game
    #if (range1 >= 200 and range1 <= 330):
    if (signal_range >= 170 and signal_range <= 410 and signal_out_range < signal_range/2):
        print ("blink")
    elif (signal_range >= 300 and signal_range <= 600 and signal_out_range > signal_range*2/3):
        print ("bite") 
    return ""


def start():
    sfreq = 256
    host = "localhost"
    epoch_size_in_seconds = 1

    channels = [
        "Fp1",
        "Fp2",
        "F3",
        "F4",
        "C1",
        "C3",
        "C2",
        "C4",
        "CP1",
        "CP3",
        "CP2",
        "CP4",
        "Cz",
        "O1",
        "O2",
        "Pz",
    ]

    stream_info = mne.create_info(
        ch_names=channels, sfreq=sfreq, ch_types="eeg", montage="standard_1020"
    )


    with LSLClient(info=stream_info, host=host) as client:
        client_info = client.get_measurement_info()
        while True:
            epoch = client.get_data_as_epoch(n_samples=epoch_size_in_seconds * sfreq)
            #send_server_ok_signal()
            process_epoch(epoch)
            time.sleep(1)


#def send_server_ok_signal():
#    sse.publish("ok", type="status")


#def send_blink_artifact_event():
#    sse.publish("blink", type="artifact")

In [5]:
start()

Client: Waiting for server to start


<ipython-input-4-4b8d31581b0c>:58: DeprecationWarning: Passing montage to create_info is deprecated and will be removed in 0.21, use raw.set_montage (or epochs.set_montage, etc.) instead
  ch_names=channels, sfreq=sfreq, ch_types="eeg", montage="standard_1020"


Client: Connected
1 matching events found
No baseline correction applied
Not setting metadata
0 projection items activated
0 bad epochs dropped
average range= 44.60986328125
1 matching events found
No baseline correction applied
Not setting metadata
0 projection items activated
0 bad epochs dropped
average range= 332.3149719238281
blink
1 matching events found
No baseline correction applied
Not setting metadata
0 projection items activated
0 bad epochs dropped
average range= 34.291656494140625
1 matching events found
No baseline correction applied
Not setting metadata
0 projection items activated
0 bad epochs dropped
average range= 323.9132385253906
blink
1 matching events found
No baseline correction applied
Not setting metadata
0 projection items activated
0 bad epochs dropped
average range= 42.20361328125
1 matching events found
No baseline correction applied
Not setting metadata
0 projection items activated
0 bad epochs dropped
average range= 319.602783203125
blink
1 matching event

In [8]:
raw.get_data()

array([[ -7638.39335757,  -7644.61105065,  -7651.09643305, ...,
         -4143.49013451,  -4143.81866232,  -4136.98041671],
       [  3128.37569166,   3120.81955194,   3112.80103973, ...,
          5145.26878164,   5143.55313639,   5151.66899017],
       [-28749.72454398, -28757.2928514 , -28761.29602365, ...,
        -25648.16645611, -25650.81901402, -25640.75632875],
       ...,
       [-39953.21656758, -39964.77587956, -39941.86410644, ...,
        -39597.23842933, -39623.93435616, -39632.40307314],
       [-24932.21917505, -24922.58235917, -24925.83113422, ...,
        -26110.58760617, -26149.67024837, -26155.48640745],
       [ -9674.77909786,  -9678.52674848,  -9679.92603361, ...,
         -8830.62079537,  -8834.59963223,  -8830.1462552 ]])

In [9]:
average_max_signal=(np.amax(raw[0][0])+np.amax(raw[1][0])+np.amax(raw[2][0])+np.amax(raw[3][0]))/4
average_min_signal=(np.amin(raw[0][0])+np.amin(raw[1][0])+np.amin(raw[2][0])+np.amin(raw[3][0]))/4
signal_range = average_max_signal-average_min_signal
print (signal_range)

8805.6496395675


In [13]:
np.amax(raw.get_data()[0][0])

-7638.3933575701

In [37]:
print ("max 1: " + str(88))

max 1: 88


# Let's observe it
plt.ion()  # make plot interactive
_, ax = plt.subplots(1)

with LSLClient(info=raw.info, host=host, wait_max=wait_max) as client:
    client_info = client.get_measurement_info()
    sfreq = int(client_info['sfreq'])
    #print(client_info)

    # let's observe ten seconds of data
    for ii in range(20):
        plt.cla()
        epoch = client.get_data_as_epoch(n_samples=sfreq)
        #epoch.plot_psd_topomap()
        #print(epoch.get_data())
        #epoch.average().plot(axes=ax)
        plt.pause(1)
#plt.draw()

me llega el lsl en formato epoc, no en formato raw. Hay que ajustar el código para que epoch se convierta en raw
usar funcion epoch.get_data()